This notebook computes the Frost Days index: starting from the daily minimum temperature (1980-1990) TN,
the Frost Days index is annual cound of days when TN (daily minimum temperature) < 0°C

Connect to the remote ECAS instance

In [1]:
from PyOphidia import cube
cube.Cube.setclient('k204213', 'abcd', 'ecas-server.dkrz.de', '11732')

Current cdd is /
Current session is http://ecas-server.dkrz.de/ophidia/sessions/16084439523705251231526460832420091/experiment
Current cwd is /
The last produced cube is http://ecas-server.dkrz.de/ophidia/25/671


Import source data (minimum temperature °K)

In [2]:
mintemp = cube.Cube(src_path='/usr/local/ophidia/ECAS/data/repository/E63_kp40_198001_199012_T2m_daymin_merged.nc',
    measure='temp2',
    import_metadata='yes',
    imp_dim='time',
    imp_concept_level='d', vocabulary='CF',hierarchy='oph_base|oph_base|oph_time',
    ncores=4,
    description='Min Temps'
    )

New cube is http://ecas-server.dkrz.de/ophidia/14/992


Identify the frost days

In [3]:
frostdays = mintemp.apply(
    query="oph_predicate('OPH_FLOAT','OPH_INT',measure,'x-273.15','<0','1','0')"
)

Count the number of frost days

In [4]:
count = frostdays.reduce2(
    operation='sum',
    dim='time',
    concept_level='y',
)

Plot the result

In [5]:
firstyear = count.subset(subset_filter=1, subset_dims='time')

In [6]:
data = firstyear.export_array(show_time='yes')

In [7]:

import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, cm
import numpy as np

data = firstyear.export_array(show_time='yes')
lat = data['dimension'][0]['values'][:]
lon = data['dimension'][1]['values'][:]
var = data['measure'][0]['values'][:]
var = np.reshape(var, (len(lat), len(lon)))

fig = plt.figure(figsize=(15, 15), dpi=100)
ax  = fig.add_axes([0.1,0.1,0.8,0.8])

map = Basemap(projection='cyl',llcrnrlat= -90,urcrnrlat= 90, llcrnrlon=0,urcrnrlon=360, resolution='c')


map.drawcoastlines()
map.drawparallels(np.arange( -90, 90,30),labels=[1,0,0,0])
map.drawmeridians(np.arange(-180,180,30),labels=[0,0,0,1])


x, y = map(*np.meshgrid(lon,lat))

clevs = np.arange(0,371,10)
cnplot = map.contourf(x,y,var,clevs,cmap=plt.cm.jet)
cbar = map.colorbar(cnplot,location='right')

plt.title('Frost Days (year 1980)')
plt.show()


<Figure size 1500x1500 with 2 Axes>